In [1]:
from pymatgen.core import Structure
from joblib import Memory
from mp_api.client import MPRester
from ase.visualize import view

memory = Memory('.cachedir')

In [2]:
def view_structure(structure, viewer='ase'):
    return view(structure.to_ase_atoms(), viewer=viewer)


In [3]:
@memory.cache
def get_structure(mp_id):
    with MPRester() as mpr:
        # docs = mpr.materials.summary.search(material_ids=[mp_id], fields=["structure"])
        # structure = docs[0].structure
        # # -- Shortcut for a single Materials Project ID:
        structure = mpr.get_structure_by_material_id(mp_id,)
    
    return structure

structure = get_structure('mp-1265')
view_structure(structure)

<Popen: returncode: None args: ['/Users/tw/miniforge3/envs/surfpes/bin/pytho...>

In [12]:
from pymatgen.core.surface import generate_all_slabs
# from mp_api.client import MPRester


# structure = get_structure('mp-1265')
slabs = generate_all_slabs(structure.to_conventional(), 1, 10, 20, include_reconstructions=False, primitive=True, max_normal_search=6,)
# there is an apparent mismatch between orientated unit cell and the slab if primitive=True, max_normal_search=None
# this doesn't occur if primitive=True, max_normal_search=5
# What is happening? Not sure!
# /Users/tw/miniforge3/envs/surfpes/lib/python3.11/site-packages/pymatgen/core/surface.py#1156
print(len(slabs))

3


In [13]:
import ipywidgets as widgets
from IPython.display import clear_output

def show_slab(idx):
    global i
    i = idx
    clear_output(wait=True)
    print(slabs[i].reconstruction)
    print(slabs[i])
    return view_structure(slabs[i] * (2, 2, 1), viewer='x3d')

widgets.interact(
    show_slab,
    idx=widgets.IntSlider(value=0, min=0, max=len(slabs)-1, step=1, description="i")
);

interactive(children=(IntSlider(value=0, description='i', max=2), Output()), _dom_classes=('widget-interact',)…

## Lattice Matching

In [18]:
from pymatgen.analysis.interfaces.substrate_analyzer import SubstrateAnalyzer

In [22]:
with MPRester() as mpr:
    docs = mpr.materials.summary.search(
        # elements=[element]
        material_ids=["mp-149", "mp-13", "mp-22526"]
    )

Retrieving SummaryDoc documents:   0%|          | 0/3 [00:00<?, ?it/s]

In [60]:
from pymatgen.core import Element
from pathlib import Path
from monty.serialization import loadfn, dumpfn

In [92]:
mp_data_path = Path('mp_data'); mp_data_path.mkdir(exist_ok=True)

def get_elemental_data():
    element_to_summary = {}

    for i in range(1, 86):
        el = Element.from_Z(i)
        el_path = mp_data_path/str(el); el_path.mkdir(exist_ok=True)

        if el.is_metal:
            summary_path = el_path/'summary.json.gz'
            if summary_path.exists():
                element_to_summary[el] = loadfn(summary_path)
            else:
                with MPRester() as mpr:
                    docs = mpr.materials.summary.search(
                        elements=[str(el)],
                        num_elements=1,
                        is_stable=True,
                    )
                if len(docs) == 0:
                    with MPRester() as mpr:
                        docs = sorted(mpr.materials.summary.search(
                            elements=[str(el)],
                            num_elements=1,
                            is_stable=False,
                        ), key=lambda d: d.formation_energy_per_atom)
                        docs = [docs[0]]
                assert len(docs) == 1, f"Expected 1 summary document for {el}, got {len(docs)}"
                dumpfn(docs[0].dict(), summary_path)
                element_to_summary[el] = docs[0]
    return element_to_summary

def get_elasticity_data(element_to_summary):
    element_to_elasticity = {}
    for el, summary in element_to_summary.items():
        elastic_path = mp_data_path/str(el)/'elastic.json.gz'
        if elastic_path.exists():
            element_to_elasticity[el] = loadfn(elastic_path)
        else:
            with MPRester() as mpr:
                docs = mpr.materials.elasticity.search(material_ids=[summary.get('material_id')],)
            if len(docs) == 0:
                print(f"No elasticity data for {el} ({summary['material_id']})")
                continue
                
            assert len(docs) == 1, f"Expected 1 elasticity document for {el} ({summary['material_id']}), got {len(docs)}"
            dumpfn(docs[0].dict(), elastic_path)
            element_to_elasticity[el] = docs[0]
    return element_to_elasticity
 

el_to_summary = get_elemental_data()      
el_to_elasticity = get_elasticity_data(el_to_summary)      

Retrieving ElasticityDoc documents: 0it [00:00, ?it/s]

No elasticity data for K (mp-1184804)


Retrieving ElasticityDoc documents: 0it [00:00, ?it/s]

No elasticity data for Mn (mp-35)


Retrieving ElasticityDoc documents: 0it [00:00, ?it/s]

...

Retrieving ElasticityDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElasticityDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElasticityDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElasticityDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElasticityDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [56]:
from ase.data import atomic_numbers

metal_substrate_elements = ['']

for element in atomic_numbers:
    if element == 'X': continue
    with MPRester() as mpr:
        docs = mpr.materials.summary.search(
            elements=['Ni'],
            num_elements=1,
            is_stable=True,
        )
    break

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [57]:
docs

[MPDataDoc<SummaryDoc>(
 builder_meta=EmmetMeta(emmet_version='0.84.3rc4', pymatgen_version='2024.11.13', run_id='32bfb79c-5ce0-41ab-ab69-69ba9fb96205', batch_id=None, database_version='2025.09.25', build_date=datetime.datetime(2024, 11, 21, 21, 59, 41, 903000), license='BY-C'),
 nsites=1,
 elements=[Element Ni],
 nelements=1,
 composition=Composition('Ni1'),
 composition_reduced=Composition('Ni1'),
 formula_pretty='Ni',
 formula_anonymous='A',
 chemsys='Ni',
 volume=10.492020341068946,
 density=9.289219853118466,
 density_atomic=10.492020341068946,
 symmetry=SymmetryData(crystal_system=<CrystalSystem.cubic: 'Cubic'>, symbol='Fm-3m', number=225, point_group='m-3m', symprec=0.1, angle_tolerance=5.0, version='2.5.0'),
 property_name='summary',
 material_id=MPID(mp-23),
 deprecated=False,
 deprecation_reasons=None,
 last_updated=datetime.datetime(2025, 9, 23, 22, 39, 50, 172000),
 origins=[PropertyOrigin(name='structure', task_id=MPID(mp-1949183), last_updated=datetime.datetime(2021, 3, 5

In [53]:
docs

[MPDataDoc<ElasticityDoc>(
 builder_meta=EmmetMeta(emmet_version='0.84.6rc5', pymatgen_version='2024.11.13', run_id='594815f4-e1b1-4ff4-9598-f5e9f2d2cc26', batch_id=None, database_version='2025.09.25', build_date=datetime.datetime(2025, 3, 20, 19, 25, 25, 914000), license=None),
 nsites=1,
 elements=[Element Cs],
 nelements=1,
 composition=Composition('Cs1'),
 composition_reduced=Composition('Cs1'),
 formula_pretty='Cs',
 formula_anonymous='A',
 chemsys='Cs',
 volume=116.55740584201097,
 density=1.8934420662343912,
 density_atomic=116.55740584201097,
 symmetry=SymmetryData(crystal_system=<CrystalSystem.cubic: 'Cubic'>, symbol='Im-3m', number=229, point_group='m-3m', symprec=0.1, angle_tolerance=5.0, version='2.6.0'),
 property_name='elasticity',
 material_id=MPID(mp-1),
 deprecated=False,
 deprecation_reasons=None,
 last_updated=datetime.datetime(2025, 3, 20, 19, 25, 25, 914000),
 origins=[],
 warnings=['Small modulus. voigt bulk modulus is 1.974; smaller than 2.0.', 'Small modulus. re

In [42]:
docs = sorted(docs, key=lambda x: x.energy_per_atom)

In [ ]:
docs[0]

MPDataDoc<SummaryDoc>(
builder_meta=EmmetMeta(emmet_version='0.84.3rc4', pymatgen_version='2024.11.13', run_id='32bfb79c-5ce0-41ab-ab69-69ba9fb96205', batch_id=None, database_version='2025.09.25', build_date=datetime.datetime(2024, 11, 21, 20, 2, 48, 105000), license='BY-C'),
nsites=4,
elements=[Element Ac, Element Ir, Element Ni],
nelements=3,
composition=Composition('Ac2 Ni1 Ir1'),
composition_reduced=Composition('Ac2 Ni1 Ir1'),
formula_pretty='Ac2NiIr',
formula_anonymous='ABC2',
chemsys='Ac-Ir-Ni',
volume=100.33145455665266,
density=11.66664295683749,
density_atomic=25.082863639163165,
symmetry=SymmetryData(crystal_system=<CrystalSystem.cubic: 'Cubic'>, symbol='Fm-3m', number=225, point_group='m-3m', symprec=0.1, angle_tolerance=5.0, version='2.5.0'),
property_name='summary',
material_id=MPID(mp-862849),
deprecated=False,
deprecation_reasons=None,
last_updated=datetime.datetime(2025, 9, 23, 22, 49, 14, 579000),
origins=[PropertyOrigin(name='structure', task_id=MPID(mp-2039927), last

In [ ]:
[doc.s for doc in docs]

AttributeError: 'MPDataDoc' object has no attribute 'energy'